## **Business Case - Dashboard de Acompanhamento de Alunos**

### Códigos DAX utilizados nas medidas do Dashboard

#### Introdução

Este notebook contém uma explicação detalhada das fórmulas DAX utilizadas no desenvolvimento do **Dashboard de Acompanhamento de Alunos** no Power BI.


### **1. Índice de Massa Corporal (IMC)**
#### Descrição:
O IMC é uma medida utilizada para avaliar se o peso da pessoa está adequado à sua altura.

In [ ]:
IMC = 
IF(
    HASONEVALUE(Cadastro[Aluno]),
    DIVIDE(
        SUM('Avaliações'[Peso (kg)]),
        (SUM(Cadastro[Altura]) * SUM(Cadastro[Altura]))
    ),
    BLANK()
)

### **2. Classificação do IMC**
#### Descrição:
Código para classificar se o aluno está abaixo do peso, peso normal, acima do peso ou com obesidade, com base no IMC calculado.

In [ ]:
Classificação IMC = 
VAR IMC_Value = 
    IF(
        HASONEVALUE(Cadastro[Aluno]),
        DIVIDE(
            SUM('Avaliações'[Peso (kg)]),
            (SUM(Cadastro[Altura]) * SUM(Cadastro[Altura]))
        ),
        BLANK()
    )
RETURN
    SWITCH(
        TRUE(),
        ISBLANK(IMC_Value), "Selecione um único aluno",
        IMC_Value < 18.5, "Abaixo do Peso",
        IMC_Value >= 18.5 && IMC_Value < 24.9, "Peso Normal",
        IMC_Value >= 25 && IMC_Value < 29.9, "Sobrepeso",
        IMC_Value >= 30, "Obesidade",
        "Dados Inválidos"
    )

### **3. Razão Cintura-Quadril (RCQ)**
#### Descrição:
A relação cintura-quadril (RCQ) é o cálculo que se faz a partir das medidas da cintura e do quadril para verificar o risco que uma pessoa tem de desenvolver uma doença cardiovascular. Isso acontece porque, quanto maior a concentração da gordura abdominal, maior o risco de ter problemas como colesterol alto, diabetes, pressão alta ou aterosclerose.

In [ ]:
RCQ = 

    IF(
        HASONEVALUE(Cadastro[Aluno]),
        DIVIDE(
            SUM('Avaliações'[Medida Cintura (cm)]),
            SUM('Avaliações'[Medida Quadril (cm)])
        ),
        BLANK()
    )

### **4. Classificação do RCQ**
#### Descrição:
Código para classificar o risco que o aluno tem de desenvolver doença cardiovascular. Os valores são diferentes entre homens e mulheres.

In [ ]:
Classificação RCQ = 
VAR RQC_Value = 
    IF(
        HASONEVALUE(Cadastro[Aluno]),
        DIVIDE(
            SUM('Avaliações'[Medida Cintura (cm)]),
            SUM('Avaliações'[Medida Quadril (cm)])
        ),
        BLANK()
    )
VAR Genero_Aluno = SELECTEDVALUE(Cadastro[Sexo])
RETURN
    SWITCH(
        TRUE(),
        ISBLANK(RQC_Value), "Selecione um único aluno",
        Genero_Aluno = "Masculino",
            SWITCH(
                TRUE(),
                RQC_Value < 0.95, "Baixo",
                RQC_Value >= 0.95 && RQC_Value < 1, "Moderado",
                RQC_Value >= 1, "Alto",
                "Dados Inválidos"
            ),
        Genero_Aluno = "Feminino",
            SWITCH(
                TRUE(),
                RQC_Value < 0.8, "Baixo",
                RQC_Value >= 0.8 && RQC_Value < 0.85, "Moderado",
                RQC_Value >= 0.85, "Alto",
                "Dados Inválidos"
            ),
        "Gênero Não Definido"
    )


### **5. Percentual de Gordura Corporal - Método Pollack 7 Dobras**

#### Descrição:
O método Pollack de 7 dobras calcula o percentual de gordura corporal com base na medição da espessura da pele em sete locais do corpo: peitoral, abdominal, coxa, axilar média, supra-ilíaca, subescapular e tríceps. A fórmula utiliza uma condição para aplicar o cálculo correto, baseado no gênero (masculino ou feminino).


In [ ]:
% Gordura = 
VAR Densidade_Homem = 
    IF(
        SELECTEDVALUE(Cadastro[Sexo]) = "Masculino",
        1.112 - (0.00043499 * [Somatória 7 Dobras]) + 
        (0.00000055 * ([Somatória 7 Dobras] ^ 2)) - 
        (0.00028826 * SELECTEDVALUE(Cadastro[Idade])),
        BLANK()
    )
VAR Densidade_Mulher = 
    IF(
        SELECTEDVALUE(Cadastro[Sexo]) = "Feminino",
        1.097 - (0.00046971 * [Somatória 7 Dobras]) + 
        (0.00000056 * ([Somatória 7 Dobras] ^ 2)) - 
        (0.00012828 * SELECTEDVALUE(Cadastro[Idade])),
        BLANK()
    )
VAR Densidade = 
    IF(
        SELECTEDVALUE(Cadastro[Sexo]) = "Masculino",
        Densidade_Homem,
        Densidade_Mulher
    )
RETURN
    IF(
        NOT(ISBLANK(Densidade)),
        ((4.95 / Densidade) - 4.5) * 100,
        BLANK()
    )
